<a href="https://colab.research.google.com/github/netaamz/cloudTut6/blob/main/pythongame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

installations



In [33]:
!pip install firebase 
!pip install Jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#imports and drive connect
import IPython
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import HTML
from jinja2 import Template
%cd /content/drive/MyDrive/pygame/
with open("styles.css",'r') as f:
  css_content=f.read()
with open("index.html",'r') as f:
  html_content=f.read()
with open("index.js",'r') as f:
  js_content=f.read()
template=Template(html_content)
html=template.render(styles=css_content,script=js_content)
display(HTML(html))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/pygame


In [ ]:


html_code = \
'''
<!DOCTYPE html>
<html>
<head>
   <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">
    <title>Quiz</title>
      <script src="https://drive.google.com/uc?export=download&id=1gtSP_dhC_RrCq8iPaylCY11NHw-NvDwH"></script>
      
</head>
<body onload="NextQuestion(0)">
    <main>
        <div id="timer">Time left: 10 seconds</div>
            <div id="life-container">
                <span class="life-icon"></span>
                <span class="life-icon"></span>
             <span class="life-icon"></span>
                </div>
        <!-- creating a modal for when quiz ends -->
        <div class="modal-container" id="score-modal">
          
            <div class="modal-content-container">
              
                <h1>Congratulations, Quiz Completed.</h1>
              
                <div class="grade-details">
                    <p>Attempts : 10</p>
                    <p>Wrong Answers : <span id="wrong-answers"></span></p>
                    <p>Right Answers : <span id="right-answers"></span></p>
                    <p>Grade : <span id="grade-percentage"></span>%</p>
                    <p ><span id="remarks"></span></p>
                </div>
              
                <div class="modal-button-container">
                    <button onclick="closeScoreModal()">Continue</button>
                </div>
              
            </div>
            
        </div>
        

        <div class="game-quiz-container">
          
            <div class="game-details-container">
                <h1>Score : <span id="player-score"></span> / 10</h1>
                <h1> Question : <span id="question-number"></span> / 10</h1>
            </div>

            <div class="game-question-container">
                <h1 id="display-question"></h1>
            </div>

            <div class="game-options-container">
              
               <div class="modal-container" id="option-modal">
                 
                    <div class="modal-content-container">
                         <h1>Please Pick An Option</h1>
                      
                         <div class="modal-button-container">
                            <button onclick="closeOptionModal()">Continue</button>
                        </div>
                      
                    </div>
                 
               </div>
              
                <span>
                    <input type="radio" id="option-one" name="option" class="radio" value="optionA" />
                    <label for="option-one" class="option" id="option-one-label"></label>
                </span>
              

                <span>
                    <input type="radio" id="option-two" name="option" class="radio" value="optionB" />
                    <label for="option-two" class="option" id="option-two-label"></label>
                </span>
              

                <span>
                    <input type="radio" id="option-three" name="option" class="radio" value="optionC" />
                    <label for="option-three" class="option" id="option-three-label"></label>
                </span>
              

                <span>
                    <input type="radio" id="option-four" name="option" class="radio" value="optionD" />
                    <label for="option-four" class="option" id="option-four-label"></label>
                </span>


            </div>

            <div class="next-button-container">
                <button onclick="handleNextQuestion()">Next Question</button>
                <button id="exit-button" onclick="confirmExit()">Exit Game</button>
            </div>
            
        </div>
             
    </main>

</body>
</html>

'''
display(IPython.display.HTML(html_code))

In [34]:
from IPython.display import display, HTML, clear_output
import google.colab.output
from firebase import firebase
##imports and drive connect
import IPython
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import HTML
from jinja2 import Template
%cd /content/drive/MyDrive/pygame/
#with open("styles.css",'r') as f:
 # css_content=f.read()
#with open("index.html",'r') as f:
 # html_content=f.read()
#with open("index.js",'r') as f:
  #script=f.read()
  #template=Template(html_content)
  #html=template.render(styles=css_content,script=script)
  ##display(IPython.display.HTML(html))
FBconn = firebase.FirebaseApplication('https://pythongame-a4fc4-default-rtdb.firebaseio.com/', None)

def loadQuestions():
  questions = []
  res = FBconn.get('/python_questions/',None)
  for question in res:
    questions.append(question)
  return questions
questions=loadQuestions()

html_loginFailed = '''
        <h1 id="login-failed">Login Failed. Please try again.</h1>
        '''
html_login = '''
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">
    <script>
        function login() {
            var username = document.getElementById('username').value;
            var password = document.getElementById('password').value;
            
            // Send the username and password to the Python backend
            google.colab.kernel.invokeFunction('handle_login', [username, password], {});
        }
    </script>
</head>
<body>
    <div id="login-screen">
        <div class="login-container">
            <h1>Login</h1>
            <label for="username">Username:</label>
            <input type="text" id="username" name="username" placeholder="Enter your username">
            <label for="password">Password:</label>
            <input type="password" id="password" name="password" placeholder="Enter your password">
            <input type="button" class="login-btn" value="Login" onclick="login()">
        </div>
    </div>
</body>
</html>

'''
html_mainScreen = '''
<!DOCTYPE html>
<html>
<head>
  <title>Main Menu</title>
 <link rel="stylesheet" type="text/css" href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT">
  <script>
    function logout() {
      google.colab.kernel.invokeFunction('handle_logout');
    }

    function showPopup() {
      var popupContainer = document.getElementById('popup-container');
      popupContainer.style.display = 'block';
    }

    function closePopup() {
      var popupContainer = document.getElementById('popup-container');
      popupContainer.style.display = 'none';
    }
    function openQuestions(){
      google.colab.kernel.invokeFunction('handle_quiz');
    }
  </script>
</head>
<body>
<div id="menu" class="container">
    <h1>Python Quiz</h1>
    <div class="menu">
      <button class="button help-button" onclick="showPopup()">?</button>
      <button class="button" onclick="openQuestions()">Play</button>
      <button class="button">My Score</button>
      <button class="button">Ranking</button>
    </div>
    <button class="button logout-button" onclick="logout()">Logout</button>
</div>

  <div id="popup-container" class="popup-container">
    <div class="popup">
      <h2>Help</h2>
      <p>This is the help section. Here you can find information and instructions on how to use the Python Quiz application.</p>
      <button class="button close-button" onclick="closePopup()">Close</button>
    </div>
  </div>
</body>
</html>
'''

html_questions='''
<!DOCTYPE html>
<html>
  <head>
    <!DOCTYPE link rel="stylesheet" href="styleSheetgame.css">
    <link
      rel="stylesheet"
      type="text/css"
      href="https://drive.google.com/uc?id=1WQCIPgq9rt5VkLC2H89zB_-esW7iaLGT"/>

    <title>Quiz</title>
    <script>
          var lives = 3
          let shuffledQuestions = {{questions}}; //empty array to hold shuffled selected questions
          console.log(shuffledQuestions)
          function handleQuestions() {
        //function to shuffle and push 10 questions to shuffledQuestions array
            while (shuffledQuestions.length <= 9) {
              const random = questions[Math.floor(Math.random() * questions.length)];
              if (!shuffledQuestions.includes(random)) {
                  shuffledQuestions.push(random);
          }
        }
      }
      let questionNumber = 1;
      let playerScore = 0;
      let wrongAttempt = 0;
      let indexNumber = 0;
      let lives = 3;
      // function for displaying next question in the array to dom
      function NextQuestion(index) {
        handleQuestions();
         timeLeft = 10; // reset the timer to 10 seconds
         clearTimeout(timerId); // reset the timerId to prevent multiple timers running at the same time
         timerId = setTimeout(countdown, 1000); // start the countdown timer
        document.getElementById("timer").innerHTML = "Time left: " + timeLeft + " seconds"; // update the timer display
        const currentQuestion = shuffledQuestions[index];
        document.getElementById("question-number").innerHTML = questionNumber;
        document.getElementById("player-score").innerHTML = playerScore;
        document.getElementById("display-question").innerHTML =
          currentQuestion.question;
        document.getElementById("option-one-label").innerHTML =
          currentQuestion.optionA;
        document.getElementById("option-two-label").innerHTML =
          currentQuestion.optionB;
        document.getElementById("option-three-label").innerHTML =
          currentQuestion.optionC;
        document.getElementById("option-four-label").innerHTML =
          currentQuestion.optionD;
      }



      function checkForAnswer() {
        const currentQuestion = shuffledQuestions[indexNumber]["Question"];
        const currentQuestionAnswer = currentQuestion["Correct"];
        const options = [currentQuestion["Option1"],currentQuestion["Option2"],currentQuestion["Option3"],currentQuestion["Option4"]];
        let correctOption = null;

        options.forEach((option) => {
          if (option.value === currentQuestionAnswer) {
            correctOption = option.labels[0].id;
          }
        });

        if (
          options[0].checked === false &&
          options[1].checked === false &&
          options[2].checked === false &&
          options[3].checked === false
        ) {
          console.log("Please select an option.");
          return;
        }

        options.forEach((option) => {
          if (option.checked === true && option.value === currentQuestionAnswer) {
            document.getElementById(correctOption).style.backgroundColor = "green";
            playerScore++;
            indexNumber++;
            setTimeout(() => {
              questionNumber++;
            }, 1000);
          } else if (option.checked && option.value !== currentQuestionAnswer) {
            const wrongLabelId = option.labels[0].id;
            document.getElementById(wrongLabelId).style.backgroundColor = "red";
            document.getElementById(correctOption).style.backgroundColor = "green";
            wrongAttempt++;
            indexNumber++;
            setTimeout(() => {
              questionNumber++;
            }, 1000);
            lives--;
            updateLives(lives);
          }
        });
      }

      function handleNextQuestion() {
        checkForAnswer();
        unCheckRadioButtons();
        setTimeout(() => {
          if (indexNumber <= 9) {
            NextQuestion(indexNumber);
          } else {
            handleEndGame();
          }
          resetOptionBackground();
        }, 1000);
      }

      function resetOptionBackground() {
        const options = document.getElementsByName("option");
        options.forEach((option) => {
          document.getElementById(option.labels[0].id).style.backgroundColor = "";
        });
      }

      function unCheckRadioButtons() {
        const options = document.getElementsByName("option");
        options.forEach((option) => {
          option.checked = false;
        });
      }

      function handleEndGame() {
        let remark = null;
        let remarkColor = null;

        if (playerScore <= 3) {
          remark = "Bad Grades, Keep Practicing.";
          remarkColor = "red";
        } else if (playerScore >= 4 && playerScore < 7) {
          remark = "Average Grades, You can do better.";
          remarkColor = "orange";
        } else if (playerScore >= 7) {
          remark = "Excellent, Keep the good work going.";
          remarkColor = "green";
        }
        const playerGrade = (playerScore / 10) * 100;

        console.log("Remarks:", remark);
        console.log("Player Grade:", playerGrade);
        console.log("Wrong Answers:", wrongAttempt);
        console.log("Right Answers:", playerScore);
      }

      function updateLives(lives) {
        const lifeContainer = document.getElementById("life-container");

        while (lifeContainer.firstChild) {
          lifeContainer.removeChild(lifeContainer.firstChild);
        }

      for (let i = 0; i < lives; i++) {
          const lifeIcon = document.createElement("span");
          lifeIcon.classList.add("life-icon");
          lifeContainer.appendChild(lifeIcon);
        }

        if (lives === 0) {
          handleEndGame();
        }
      }

      let exitModal = document.getElementById("exit-modal");

      function handleExitGame() {
        exitModal.style.display = "flex";
      }

      function closeExitModal() {
        exitModal.style.display = "none";
      }

      function exitGame() {
        window.location.reload();
      }

      function confirmExit() {
        console.log("Exiting the game...");
      }

      // Add the countdown function

      let timeLeft = 50;
      let timerId;

      function countdown() {
        if (timeLeft === 0) {
          clearTimeout(timerId);
          console.log("Time's up!");
        } else {
          console.log("Time left:", timeLeft, "seconds");
          timeLeft--;
          timerId = setTimeout(countdown, 1000);
        }
      }
    </script>
  </head>
  <body>
    <main>
      <div id="timer">Time left: 10 seconds</div>
      <div id="life-container">
        <img
          src="https://drive.google.com/uc?id=1wusZvxqsBuHcRTB-6ZYmFWzkmjA2jSoQ"
          class="life-icon"
        />
        <img
          src="https://drive.google.com/uc?id=1wusZvxqsBuHcRTB-6ZYmFWzkmjA2jSoQ"
          class="life-icon"
        />
        <img
          src="https://drive.google.com/uc?id=1wusZvxqsBuHcRTB-6ZYmFWzkmjA2jSoQ"
          class="life-icon"
        />
      </div>
      <!-- creating a modal for when quiz ends -->
      <div class="modal-container" id="score-modal">
        <div class="modal-content-container">
          <h1>Congratulations, Quiz Completed.</h1>

          <div class="grade-details">
            <p>Attempts : 10</p>
            <p>Wrong Answers : <span id="wrong-answers"></span></p>
            <p>Right Answers : <span id="right-answers"></span></p>
            <p>Grade : <span id="grade-percentage"></span>%</p>
            <p><span id="remarks"></span></p>
          </div>

          <div class="modal-button-container">
            <button onclick="closeScoreModal()">Continue</button>
          </div>
        </div>
      </div>

      <div class="game-quiz-container">
        <div class="game-details-container">
          <h1>Score : <span id="player-score"></span> / 10</h1>
          <h1>Question : <span id="question-number"></span> / 10</h1>
        </div>

        <div class="game-question-container">
          <h1 id="display-question"></h1>
        </div>

        <div class="game-options-container">
          <div class="modal-container" id="option-modal">
            <div class="modal-content-container">
              <h1>Please Pick An Option</h1>

              <div class="modal-button-container">
                <button onclick="closeOptionModal()">Continue</button>
              </div>
            </div>
          </div>

          <span>
            <input
              type="radio"
              id="option-one"
              name="option"
              class="radio"
              value="optionA"
            />
            <label
              for="option-one"
              class="option"
              id="option-one-label"
            ></label>
          </span>

          <span>
            <input
              type="radio"
              id="option-two"
              name="option"
              class="radio"
              value="optionB"
            />
            <label
              for="option-two"
              class="option"
              id="option-two-label"
            ></label>
          </span>

          <span>
            <input
              type="radio"
              id="option-three"
              name="option"
              class="radio"
              value="optionC"
            />
            <label
              for="option-three"
              class="option"
              id="option-three-label"
            ></label>
          </span>

          <span>
            <input
              type="radio"
              id="option-four"
              name="option"
              class="radio"
              value="optionD"
            />
            <label
              for="option-four"
              class="option"
              id="option-four-label"
            ></label>
          </span>
        </div>

        <div class="next-button-container">
          <button onclick="handleNextQuestion()">Next Question</button>
          <button id="exit-button" onclick="confirmExit()">Exit Game</button>
        </div>
      </div>
    </main>
  </body>
</html>
'''


def isUserExist(username, password):
    users = FBconn.get('/users/', None)
    for user_id, user_data in users.items():
        if user_data.get('username') == username and user_data.get('password') == password:
            return True
    return False

def login(username, password):
    if isUserExist(username, password):
        clear_output()
        display(HTML(html_mainScreen))  # Display the quiz screen
        display(HTML('''
        <script>
            document.getElementById('login-screen').style.display = 'none';
            document.querySelector('.popup-container').style.display = 'none';
        </script>
        '''))
    else:
        html_code = '''
        <h1 id="login-failed">Login Failed. Please try again.</h1>
        '''
        display(HTML(html_code))

def logout():
    clear_output()
    display(HTML(html_login))  # Display the login screen
def openQuiz():
  clear_output()
  display(HTML(html_questions))

google.colab.output.register_callback('handle_logout', logout)
google.colab.output.register_callback('handle_login', login)
google.colab.output.register_callback('handle_quiz', openQuiz)
display(HTML(html_login))

